In [2]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

### Gathering Data

In [4]:
# stellar mass estimate output [log(M_*/M_sun)] from Bagpipes
UNICORN_path = '../data/Marissa_Stellar_Mass_CEERS.txt'
CANDLES_path = '../data/Marissa_Stellar_Mass_CANDLES.txt'
UNICORN_mass_estimates_df = pd.read_csv(UNICORN_path, delim_whitespace=True, skiprows=0)
CANDLES_mass_estimates_df = pd.read_csv(CANDLES_path, delim_whitespace=True, skiprows=0)
UNICORN_mass_estimates_df

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
9092,7.690891,7.671910,7.632544,7.682784,7.665990,7.692459,7.681779,7.581545,7.649728,7.661214,...,7.685618,7.566227,7.622362,7.658666,7.700586,7.698054,7.630552,7.696024,7.599649,7.626753
237,7.917302,8.080179,8.011846,8.014248,7.931640,7.966159,7.982721,8.021179,8.009907,7.970472,...,7.930362,7.977736,8.096601,7.951876,7.901326,7.967948,7.929160,7.959398,7.851269,8.066491
1737,8.657285,8.799079,8.510885,8.669432,8.489831,7.875290,8.689649,8.477569,7.894005,8.051442,...,8.575142,8.636147,8.712283,7.792628,8.628801,8.756745,8.888615,7.974265,7.882999,7.864168
82676,7.471130,7.495211,7.548860,7.391872,7.537369,7.351377,7.434342,7.402057,7.455115,7.562452,...,7.602916,7.390162,7.371841,7.461815,7.578313,7.481381,7.559199,7.682960,7.576630,7.464895
3353,9.576102,9.454922,9.469307,9.508620,9.539495,9.544183,9.508620,9.559758,9.500686,9.499394,...,9.445209,9.462249,9.540148,9.524289,9.537127,9.510578,9.448932,9.532774,9.525214,9.553535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14914,8.332807,8.273834,8.348857,8.494209,8.357374,8.281231,8.388155,8.404323,8.315274,8.480480,...,8.316980,8.391903,8.341746,8.240284,8.455140,8.344195,8.378230,8.279448,8.504394,8.411685
4071,8.681575,8.697223,8.745421,8.701568,8.552606,8.649651,8.553661,8.628200,8.556500,8.690540,...,8.757479,8.719186,8.728819,8.738861,8.678378,8.707144,8.764297,8.723340,8.676270,8.615550
31315,9.664033,9.483931,9.676443,9.648980,9.563239,9.601320,9.592206,9.624884,9.547973,9.605633,...,9.478650,9.699021,9.633065,9.551929,9.738364,9.567507,9.502912,9.705745,9.539392,9.486213
20853,8.891861,8.897786,8.920686,8.936282,8.987534,8.887786,8.916425,9.019704,8.907830,9.021950,...,9.028012,8.948344,8.867625,9.083784,8.939963,8.895565,8.916741,8.948583,8.940779,8.877744


In [5]:
# gathering cross matching data
IDs_UNICORN_to_NIRSpec_path = '../data/Marissa_Cross_Match_CEERS.txt'
IDs_CANDLES_to_NIRSpec_path = '../data/Marissa_Cross_Match_CANDLES.txt'
IDs_UNICORN_to_NIRSpec = pd.read_csv(IDs_UNICORN_to_NIRSpec_path, delim_whitespace=True, skiprows=0)
IDs_CANDLES_to_NIRSpec = pd.read_csv(IDs_CANDLES_to_NIRSpec_path, delim_whitespace=True, skiprows=0)
IDs_UNICORN_to_NIRSpec

,NIRSpec_ID,UNICORN_ID,Separation,Sep_Mask
0,515,9092,0.014392,True
1,618,237,0.025842,True
2,669,1737,0.042644,True
3,672,82676,0.024250,True
4,1207,3353,0.043916,True
...,...,...,...,...
127,12286,103810,43.039914,False
128,12507,103399,52.336645,False
129,16406,103629,2.589107,False
130,17496,47297,11.079998,False


In [7]:
# saving ID column
UNICORN_IDs = UNICORN_mass_estimates_df.reset_index().rename(columns={'index': 'ID'})['ID']

# calculating median, 16th percentile, and 84th percentile for each row
medians = UNICORN_mass_estimates_df.median(axis=1)
percentile_16 = UNICORN_mass_estimates_df.apply(lambda x: np.percentile(x, 16), axis=1)
percentile_84 = UNICORN_mass_estimates_df.apply(lambda x: np.percentile(x, 84), axis=1)

# calculating error from percentiles
error_16 = np.array(medians) - np.array(percentile_16)
error_84 = np.array(percentile_84) - np.array(medians)

# updating dataframe with these values
UNICORN_masses_df = pd.DataFrame({
    'UNICORN_ID': list(UNICORN_IDs),
    'MEDIAN': list(medians),
    '16% ERROR': list(error_16),
    '84% ERROR': list(error_84)
})

# saving ID column
CANDLES_IDs = CANDLES_mass_estimates_df.reset_index().rename(columns={'index': 'ID'})['ID']

# calculating median, 16th percentile, and 84th percentile for each row
medians = CANDLES_mass_estimates_df.median(axis=1)
percentile_16 = CANDLES_mass_estimates_df.apply(lambda x: np.percentile(x, 16), axis=1)
percentile_84 = CANDLES_mass_estimates_df.apply(lambda x: np.percentile(x, 84), axis=1)

# calculating error from percentiles
error_16 = np.array(medians) - np.array(percentile_16)
error_84 = np.array(percentile_84) - np.array(medians)

# updating dataframe with these values
CANDLES_masses_df = pd.DataFrame({
    'CANDLES_ID': list(CANDLES_IDs),
    'MEDIAN': list(medians),
    '16% ERROR': list(error_16),
    '84% ERROR': list(error_84)
})

# merge the two DataFrames on "UNICORN ID" and save order of NIRSpec IDs to insert into main dataframe
merged_df1 = pd.merge(UNICORN_masses_df, IDs_UNICORN_to_NIRSpec, on='UNICORN_ID', how='left')
NIRSpec_ID1 = merged_df1['NIRSpec_ID']
UNICORN_masses_df.insert(1,'NIRSpec_ID', list(NIRSpec_ID1))
UNICORN_masses_df.sort_values(by='NIRSpec_ID',ignore_index=True,inplace=True)  # sort by NIRSpec ID

# merge the two DataFrames on "UNICORN ID" and save order of NIRSpec IDs to insert into main dataframe
merged_df2 = pd.merge(CANDLES_masses_df, IDs_CANDLES_to_NIRSpec, on='CANDLES_ID', how='left')
NIRSpec_ID2 = merged_df2['NIRSpec_ID']
CANDLES_masses_df.insert(1,'NIRSpec_ID', list(NIRSpec_ID2))
CANDLES_masses_df.sort_values(by='NIRSpec_ID',ignore_index=True,inplace=True) # sort by NIRSpec ID

In [9]:
UNICORN_masses_df

,UNICORN_ID,NIRSpec_ID,MEDIAN,16% ERROR,84% ERROR
0,20546,386,8.903813,0.042291,0.112445
1,8807,397,9.372133,0.167665,0.074255
2,9092,515,7.638716,0.045778,0.059338
3,237,618,7.962204,0.045719,0.054043
4,1737,669,8.521950,0.660201,0.223057
...,...,...,...,...,...
58,27166,25856,8.660708,0.128796,0.104069
59,22793,26836,8.845213,0.021999,0.240722
60,10705,29756,7.833782,0.070545,0.225544
61,1490,31057,7.897793,0.018471,0.018080


In [13]:
CANDLES_masses_df

,CANDLES_ID,NIRSpec_ID,MEDIAN,16% ERROR,84% ERROR
0,18016,323,10.564479,0.172651,0.152230
1,19473,355,10.110639,0.334886,0.200750
2,19977,381,9.085359,0.880231,0.314829
3,20080,386,8.637563,0.081105,0.206712
4,21547,397,9.011156,0.857940,0.511539
...,...,...,...,...,...
116,28608,29756,7.478099,0.357228,0.331552
117,54269,31057,8.261649,0.370850,0.276762
118,35641,31075,8.587622,0.345762,0.146466
119,17535,36119,9.188239,0.187273,0.165053


In [15]:
sample_df = pd.read_csv('sample_Balmer_UV.csv')

# merge 'sample_df' with 'UNICORN_masses_df' on 'NIRSpec_ID'
sample_df = pd.merge(sample_df, UNICORN_masses_df[['NIRSpec_ID', 'MEDIAN', '16% ERROR', '84% ERROR']], on='NIRSpec_ID', how='left', suffixes=('', '_UNICORN'))

# merge the resulting DataFrame with 'CANDLES_masses_df' on 'NIRSpec_ID'
sample_df = pd.merge(sample_df, CANDLES_masses_df[['NIRSpec_ID', 'MEDIAN', '16% ERROR', '84% ERROR']], on='NIRSpec_ID', how='left', suffixes=('', '_CANDLES'))

# filling in NaN values within UNICORN stellar mass estimate columns with CANDLES stellar mass estimates
sample_df['MASS MEDIAN'] = sample_df['MEDIAN'].fillna(sample_df['MEDIAN_CANDLES'])
sample_df['MASS 16% ERROR'] = sample_df['16% ERROR'].fillna(sample_df['16% ERROR_CANDLES'])
sample_df['MASS 84% ERROR'] = sample_df['84% ERROR'].fillna(sample_df['84% ERROR_CANDLES'])
sample_df.drop(columns=['MEDIAN','16% ERROR','84% ERROR','MEDIAN_CANDLES','16% ERROR_CANDLES','84% ERROR_CANDLES'],inplace=True)
sample_df = sample_df.rename(columns={'NIRSpec_ID': 'NIRSPEC ID'})  # personal preference to have no underscore

In [17]:
sample_df.to_csv('sample_Balmer_UV_mass.csv',index=False)
sample_df

,NIRSPEC ID,REDSHIFT,RA,DEC,PHOTOMETRIC CATALOG,UV LUMINOSITY DENSITY REST-FRAME,UV LUMINOSITY DENSITY REST-FRAME ERROR,H-ALPHA INTEGRATED LUMINOSITY,H-ALPHA INTEGRATED LUMINOSITY MEAN ERROR,H-ALPHA INTEGRATED LUMINOSITY 16% ERROR,...,H-ALPHA INTEGRATED FLUX MEAN ERROR,H-ALPHA INTEGRATED FLUX 16% ERROR,H-ALPHA INTEGRATED FLUX 84% ERROR,H-BETA INTEGRATED FLUX,H-BETA INTEGRATED FLUX MEAN ERROR,H-BETA INTEGRATED FLUX 16% ERROR,H-BETA INTEGRATED FLUX 84% ERROR,MASS MEDIAN,MASS 16% ERROR,MASS 84% ERROR
0,323,5.667781,214.872556,52.875949,UNICORN F115W,1.983081e+44,1.716686e+43,2.590378e+42,2.012511e+41,6.623902e+41,...,5.521016e-19,1.817166e-18,2.327311e-18,3.162208e-18,8.318148e-21,2.217566e-19,2.304400e-19,10.564479,0.172651,0.152230
1,355,6.100327,214.806482,52.878827,UNICORN F115W,4.990144e+44,3.255561e+43,2.227345e+42,2.334405e+39,1.190380e+41,...,5.397559e-21,2.752371e-19,2.660095e-19,1.599096e-18,3.719649e-21,1.646765e-19,1.622306e-19,10.110639,0.334886,0.200750
2,381,5.505045,214.819671,52.879755,UNICORN F115W,1.574217e+44,2.925641e+43,1.216074e+42,3.209706e+39,1.076674e+41,...,9.423958e-21,3.161203e-19,3.309578e-19,2.529167e-19,3.850405e-20,1.381255e-19,2.123226e-19,9.085359,0.880231,0.314829
3,386,6.612528,214.832184,52.885083,UNICORN F115W,1.667184e+44,3.322362e+43,1.241948e+42,4.426539e+39,1.435680e+41,...,8.491280e-21,2.754017e-19,2.906495e-19,5.315274e-19,5.026288e-20,2.379610e-19,3.369712e-19,8.903813,0.042291,0.112445
4,397,6.000576,214.836197,52.882693,UNICORN F115W,2.550235e+45,5.405522e+43,1.080890e+43,1.538812e+39,2.141968e+41,...,3.696847e-21,5.145871e-19,5.073972e-19,9.388957e-18,1.887501e-21,3.121217e-19,3.124219e-19,9.372133,0.167665,0.074255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,29756,1.239912,215.066484,52.988621,CANDLES F606W,3.007347e+43,1.896458e+42,3.028216e+40,1.257250e+38,4.056832e+39,...,1.340741e-20,4.326236e-19,4.426340e-19,9.353249e-19,8.526022e-20,3.393824e-19,4.780012e-19,7.833782,0.070545,0.225544
124,31057,1.621038,215.069889,53.007982,CANDLES F606W,5.042346e+43,6.372160e+42,8.263794e+40,9.023853e+37,5.289918e+39,...,4.974505e-21,2.916130e-19,3.010812e-19,1.576959e-18,6.769095e-20,5.198577e-19,6.252634e-19,7.897793,0.018471,0.018080
125,31075,1.283137,215.043555,52.989917,CANDLES F606W,9.904747e+42,1.430638e+42,3.276332e+41,6.945467e+38,1.099033e+40,...,6.806629e-20,1.077064e-18,1.026404e-18,1.076887e-17,2.300299e-20,7.025824e-19,6.967026e-19,8.341897,0.235316,0.240183
126,36119,1.911995,214.930920,52.825752,CANDLES F606W,1.467381e+43,4.424978e+42,5.507181e+40,3.390945e+39,1.479400e+40,...,1.247403e-19,5.442165e-19,6.613450e-19,1.130527e-18,1.900975e-20,3.655294e-19,4.008304e-19,9.188239,0.187273,0.165053
